 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8889/notebooks/17-fullres-ang-01.ipynb#Load-libraries" data-toc-modified-id="Load-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load libraries</a></span></li><li><span><a href="http://localhost:8889/notebooks/17-fullres-ang-01.ipynb#Define-loss-functions" data-toc-modified-id="Define-loss-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define loss functions</a></span></li><li><span><a href="http://localhost:8889/notebooks/17-fullres-ang-01.ipynb#Define-models" data-toc-modified-id="Define-models-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define models</a></span></li><li><span><a href="http://localhost:8889/notebooks/17-fullres-ang-01.ipynb#Training" data-toc-modified-id="Training-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training</a></span></li></ul></div>

# Load libraries

In [1]:
import cv2
import numpy as np
import pandas as pd

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, BatchNormalization
from keras.optimizers import RMSprop
from keras.losses import binary_crossentropy
import keras.backend as K

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import math
import random
import gzip
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

# Define loss functions

In [3]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss


def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

# Define models

In [4]:
def unet_down_one_block(inputs, num_filters):
    x = Conv2D(num_filters, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [5]:
def unet_max_pool(inputs):
    x = MaxPooling2D((2, 2), strides=(2, 2))(inputs)
    return x

In [6]:
def unet_up_one_block(up_input, down_input, num_filters):
    x = UpSampling2D((2,2))(up_input)
    x = concatenate([down_input, x], axis=3)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [7]:
def get_unet(input_shape = (256, 256, 3),
             num_classes = 1,
             initial_filters = 32,
             central_filters = 1024):
    
    num_filters = initial_filters
    
    out_list    = [Input(shape=input_shape)]
    down_interim_list = []
    
    while num_filters <= central_filters/2:
        x = unet_down_one_block(out_list[-1], num_filters)
        down_interim_list.append(x)
        num_filters = num_filters * 2
        y = unet_max_pool(x)
        out_list.append(y)
    
    x = unet_down_one_block(out_list[-1], num_filters)
    out_list.append(x)
    num_filters = int(num_filters / 2)
    
    while num_filters >= initial_filters:
        x = unet_up_one_block(out_list[-1], down_interim_list.pop(), num_filters)
        out_list.append(x)
        num_filters = int(num_filters / 2)
    
    classify = Conv2D(num_classes, (1,1), activation = 'sigmoid')(out_list[-1])
    
    model = Model(inputs=out_list[0], outputs=classify)
    
    return model

In [8]:
model = get_unet(input_shape=(1152,1152,3), initial_filters=8)

In [9]:
model.load_weights('./weights/best_weights_fullres3.hdf5')

In [10]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1152, 1152, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 1152, 1152, 8) 224         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 1152, 1152, 8) 32          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 1152, 1152, 8) 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

# Training

In [11]:
df_train = pd.read_csv('data/train_masks.csv')

In [12]:
df_train.head()

,img,rle_mask
0,00087a6bd4dc_01.jpg,879386 40 881253 141 883140 205 885009 17 8850...
1,00087a6bd4dc_02.jpg,873779 4 875695 7 877612 9 879528 12 881267 15...
2,00087a6bd4dc_03.jpg,864300 9 866217 13 868134 15 870051 16 871969 ...
3,00087a6bd4dc_04.jpg,879735 20 881650 26 883315 92 883564 30 885208...
4,00087a6bd4dc_05.jpg,883365 74 883638 28 885262 119 885550 34 88716...


In [13]:
train_bboxes = pickle.load(open('./train_bboxes_resized_m512_ang_01.pkl', 'rb'))

In [14]:
actual_bboxes = pickle.load(open('./train_bboxes_actuals.pkl', 'rb'))

In [15]:
ids_train = list(train_bboxes.keys())

In [16]:
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

In [17]:
def randomHueSaturationValue(image, hue_shift_limit=(-180, 180),
                             sat_shift_limit=(-255, 255),
                             val_shift_limit=(-255, 255), u=0.5):
    if np.random.random() < u:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(image)
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = cv2.add(h, hue_shift)
        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = cv2.add(s, sat_shift)
        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = cv2.add(v, val_shift)
        image = cv2.merge((h, s, v))
        image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    return image

In [18]:
def randomShiftScaleRotate(image, mask,
                           shift_limit=(-0.0625, 0.0625),
                           scale_limit=(-0.1, 0.1),
                           rotate_limit=(-45, 45), aspect_limit=(0, 0),
                           borderMode=cv2.BORDER_CONSTANT, u=0.5):
    if np.random.random() < u:
        height, width, channel = image.shape

        angle = np.random.uniform(rotate_limit[0], rotate_limit[1])  # degree
        scale = np.random.uniform(1 + scale_limit[0], 1 + scale_limit[1])
        aspect = np.random.uniform(1 + aspect_limit[0], 1 + aspect_limit[1])
        sx = scale * aspect / (aspect ** 0.5)
        sy = scale / (aspect ** 0.5)
        dx = round(np.random.uniform(shift_limit[0], shift_limit[1]) * width)
        dy = round(np.random.uniform(shift_limit[0], shift_limit[1]) * height)

        cc = np.math.cos(angle / 180 * np.math.pi) * sx
        ss = np.math.sin(angle / 180 * np.math.pi) * sy
        rotate_matrix = np.array([[cc, -ss], [ss, cc]])

        box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
        box1 = box0 - np.array([width / 2, height / 2])
        box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

        box0 = box0.astype(np.float32)
        box1 = box1.astype(np.float32)
        mat = cv2.getPerspectiveTransform(box0, box1)
        image = cv2.warpPerspective(image, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                    borderValue=(
                                        0, 0,
                                        0,))
        mask = cv2.warpPerspective(mask, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                   borderValue=(
                                       0, 0,
                                       0,))

    return image, mask


In [19]:
def randomHorizontalFlip(image, mask, u=0.5):
    if np.random.random() < u:
        image = cv2.flip(image, 1)
        mask = cv2.flip(mask, 1)

    return image, mask


In [20]:
all_imgs  = {}
all_masks = {}
for id in ids_train:
    img  = cv2.imread('data/train/{}.jpg'.format(id))
    mask = cv2.imread('data/train_masks/{}_mask.png'.format(id), cv2.IMREAD_GRAYSCALE)
    all_imgs[id]  = img
    all_masks[id] = mask

In [21]:
def train_generator(train_batch_size):
    while True:
        this_ids_train_split = random.sample(ids_train_split, len(ids_train_split))
        for start in range(0, len(ids_train_split), train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, len(ids_train_split))
            ids_train_batch = this_ids_train_split[start:end]
            for id in ids_train_batch:
                img  = all_imgs[id]
                mask = all_masks[id]
                
                this_bbox = train_bboxes[id]
                this_bbox = [math.floor(this_bbox[0]-15), math.ceil(this_bbox[1]+15),
                             math.floor(this_bbox[2]-25), math.ceil(this_bbox[3]+25)]
                
                x_size = this_bbox[1]-this_bbox[0]+1
                y_size = this_bbox[3]-this_bbox[2]+1
                
                x_diff=1152-x_size
                y_diff=1152-y_size
                
                x_top=random.randint(0, min(x_diff, this_bbox[0]))
                x_bottom=min(x_diff-x_top, 1917-this_bbox[1])
                if (x_top+x_bottom) < x_diff:
                    if x_top==this_bbox[0]:
                        x_bottom = x_diff-x_top
                    else:
                        x_top = x_diff-x_bottom
                
                
                y_top=random.randint(0, min(y_diff, this_bbox[2]))
                y_bottom=min(y_diff-y_top, 1279-this_bbox[3])
                if (y_top+y_bottom) < y_diff:
                    if y_top==this_bbox[2]:
                        y_bottom = y_diff-y_top
                    else:
                        y_top = y_diff-y_bottom
                
                this_bbox[0] = this_bbox[0]-x_top
                this_bbox[1] = this_bbox[1]+x_bottom
                this_bbox[2] = this_bbox[2]-y_top
                this_bbox[3] = this_bbox[3]+y_bottom
                
                img  = img[this_bbox[2]:(this_bbox[3]+1), this_bbox[0]:(this_bbox[1]+1),:]
                mask = mask[this_bbox[2]:(this_bbox[3]+1), this_bbox[0]:(this_bbox[1]+1)]
                
                img = randomHueSaturationValue(img,
                                               hue_shift_limit=(-50, 50),
                                               sat_shift_limit=(-5, 5),
                                               val_shift_limit=(-15, 15))
                img, mask = randomHorizontalFlip(img, mask)
                mask = np.expand_dims(mask, axis=2)
                
                x_batch.append(img)
                y_batch.append(mask)
                
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [22]:
def valid_generator(val_batch_size):
    while True:
        for start in range(0, len(ids_valid_split), val_batch_size):
            
            x_batch = []
            y_batch = []
            
            end = min(start + val_batch_size, len(ids_valid_split))
            ids_valid_batch = ids_valid_split[start:end]
            for id in ids_valid_batch:
                img  = all_imgs[id]
                mask = all_masks[id]
                
                this_bbox = list(actual_bboxes[id])
                
                x_size = this_bbox[1]-this_bbox[0]+1
                y_size = this_bbox[3]-this_bbox[2]+1
                
                x_diff=1152-x_size
                y_diff=1152-y_size
                
                x_top=min(int(x_diff/2), this_bbox[0])
                x_bottom=min(x_diff-x_top, 1917-this_bbox[1])
                if (x_top+x_bottom) < x_diff:
                    if x_top==this_bbox[0]:
                        x_bottom = x_diff-x_top
                    else:
                        x_top = x_diff-x_bottom
                
                y_top=min(int(y_diff/2), this_bbox[2])
                y_bottom=min(y_diff-y_top, 1279-this_bbox[3])
                if (y_top+y_bottom) < y_diff:
                    if y_top==this_bbox[2]:
                        y_bottom = y_diff-y_top
                    else:
                        y_top = y_diff-y_bottom
                
                this_bbox[0] = this_bbox[0]-x_top
                this_bbox[1] = this_bbox[1]+x_bottom
                this_bbox[2] = this_bbox[2]-y_top
                this_bbox[3] = this_bbox[3]+y_bottom
                
                img  = img[this_bbox[2]:(this_bbox[3]+1), this_bbox[0]:(this_bbox[1]+1),:]
                mask = mask[this_bbox[2]:(this_bbox[3]+1), this_bbox[0]:(this_bbox[1]+1)]
                
                mask = np.expand_dims(mask, axis=2)
                
                x_batch.append(img)
                y_batch.append(mask)  
            x_batch = np.array(x_batch, np.float32) / 255
            y_batch = np.array(y_batch, np.float32) / 255
            yield x_batch, y_batch

In [23]:
train_batch_size = 6
val_batch_size   = 16

In [24]:
model.compile(optimizer=RMSprop(lr=0.001), loss=bce_dice_loss, metrics=[dice_coeff])

In [25]:
model.evaluate_generator(valid_generator(val_batch_size), np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

[0.13471121713519096, 0.96814502775669098]

In [26]:
model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=1,
                    verbose=2,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/1
192s - loss: 0.3097 - dice_coeff: 0.9141 - val_loss: 11.5321 - val_dice_coeff: 0.4708


In [27]:
max_epochs = 50

In [28]:
model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])

In [29]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               epsilon=1e-4),
             ModelCheckpoint(monitor='val_loss',
                             filepath='weights/full_res_ang_01.hdf5',
                             save_best_only=True,
                             save_weights_only=True),
             TensorBoard(log_dir='logs')]

history = model.fit_generator(generator=train_generator(train_batch_size),
                    steps_per_epoch=np.ceil(float(len(ids_train_split)) / float(train_batch_size)),
                    epochs=max_epochs,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_batch_size),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(val_batch_size)))

Epoch 1/50
185s - loss: 0.0417 - dice_coeff: 0.9830 - val_loss: 2.7337 - val_dice_coeff: 0.6881
Epoch 2/50
181s - loss: 0.0246 - dice_coeff: 0.9898 - val_loss: 0.1500 - val_dice_coeff: 0.9560
Epoch 3/50
181s - loss: 0.0179 - dice_coeff: 0.9923 - val_loss: 0.1517 - val_dice_coeff: 0.9532
Epoch 4/50
182s - loss: 0.0156 - dice_coeff: 0.9934 - val_loss: 0.0143 - val_dice_coeff: 0.9935
Epoch 5/50
182s - loss: 0.0143 - dice_coeff: 0.9940 - val_loss: 0.0142 - val_dice_coeff: 0.9935
Epoch 6/50
182s - loss: 0.0125 - dice_coeff: 0.9947 - val_loss: 0.0159 - val_dice_coeff: 0.9929
Epoch 7/50
182s - loss: 0.0116 - dice_coeff: 0.9950 - val_loss: 0.0137 - val_dice_coeff: 0.9938
Epoch 8/50
182s - loss: 0.0113 - dice_coeff: 0.9951 - val_loss: 0.0149 - val_dice_coeff: 0.9935
Epoch 9/50
182s - loss: 0.0108 - dice_coeff: 0.9954 - val_loss: 0.0118 - val_dice_coeff: 0.9946
Epoch 10/50
182s - loss: 0.0107 - dice_coeff: 0.9954 - val_loss: 0.0162 - val_dice_coeff: 0.9931
Epoch 11/50
181s - loss: 0.0102 - dice_

In [30]:
history.history

{'dice_coeff': [0.98354621337154724,
  0.98983150767529104,
  0.99239614394706066,
  0.99339472355805047,
  0.99392072703894663,
  0.99463095580499006,
  0.99500371668282461,
  0.99514923414846101,
  0.99536377520073116,
  0.99541011851603589,
  0.99562449295689737,
  0.99572897520590953,
  0.99578238800754693,
  0.99589910704319873,
  0.99591199901160288,
  0.99598228602897465,
  0.99589301657488971,
  0.9960028843616876,
  0.99610655185744523,
  0.99612079220493943,
  0.99620415610591262,
  0.99624223408736579,
  0.99599702151741565,
  0.99641556486370053,
  0.99628649921867796,
  0.99632390887718503,
  0.99640725963697663,
  0.9964243727406179,
  0.99646208042234885,
  0.99656335012180597,
  0.99664249007157457,
  0.99662288061277138,
  0.99666236047669654,
  0.99669873010455157,
  0.99665659149800701,
  0.99669558889283905,
  0.99671124661062649,
  0.99671500686585435,
  0.99672299435758216,
  0.99674935040511481,
  0.99675120894364488,
  0.9967629027178907,
  0.99677922922795215,
